Alpheios.net implementation demo
===

The Alpheios.net implementation runs on Capitains.org software suite using [Nautilus](http://github.com/capitains/nautilus).

## Configuration

The following cell are used to avoid rewriting to much cells if the address of the DTS API were to change.


In [59]:
import requests
import requests_cache

URI = "http://texts.alpheios.net/api/dts"
API_URI = "http://texts.alpheios.net"

## Getting the available endpoints

DTS entry point is a listing of the available endpoints and their URL. This means that for each implementation of DTS, this single URL will give you all the information you need to perform arbitrary queries, if you were to do so. The Alpheios Capitains implementation provide the three endpoints :

In [2]:
entry_request = requests.get(URI)
ENDPOINTS = entry_request.json()
ENDPOINTS

{'@context': 'dts/EntryPoint.jsonld',
 '@id': '/api/dts',
 '@type': 'EntryPoint',
 'collections': '/api/dts/collections',
 'documents': '/api/dts/document',
 'navigation': '/api/dts/navigation'}

As you can see, all three endpoints have been given URIs. Because we do not know the text we want to see, we'll browse from here :

## Browsing the root of the catalog

The root of the data catalog is the result of the basic GET request on the `collections` endpoint :

In [3]:
ROOT_COLLECTION  = requests.get(API_URI+ENDPOINTS["collections"]).json()
ROOT_COLLECTION

{'@context': {'@vocab': 'https://www.w3.org/ns/hydra/core#',
  'dts': 'https://w3id.org/dts/api#'},
 '@id': 'default',
 '@type': 'Collection',
 'member': [{'@id': 'urn:alpheios:greekLit',
   '@type': 'Collection',
   'title': 'Ancient Greek',
   'totalItems': 4},
  {'@id': 'urn:alpheios:latinLit',
   '@type': 'Collection',
   'title': 'Classical Latin',
   'totalItems': 1},
  {'@id': 'urn:alpheios:arabicLit',
   '@type': 'Collection',
   'title': 'Classical Arabic',
   'totalItems': 3}],
 'title': 'None',
 'totalItems': 3}

The root collection has 3 items : 1 in Ancient Greek, 1 in Classical Arabic, 1 in Classical Latin. Let's go see the one for Classical Latin.

## Requesting a specific collection

Requesting a specific collection is simple : you go to the Collections endpoint, add the parameter `id` with the `@id` property of your item.

### LatinLit

We first want to get the LatinLit collection : it will be at the URI http://texts.alpheios.net/api/dts/collections?id=urn:alpheios:latinLit

In [4]:
LatinLit  = requests.get(API_URI+ENDPOINTS["collections"]+"?id=urn:alpheios:latinLit").json()
LatinLit

{'@context': {'@vocab': 'https://www.w3.org/ns/hydra/core#',
  'dts': 'https://w3id.org/dts/api#',
  'ns1': 'http://www.w3.org/2004/02/skos/core#'},
 '@id': 'urn:alpheios:latinLit',
 '@type': 'Collection',
 'dts:extensions': {'ns1:prefLabel': [{'@language': 'eng',
    '@value': 'Classical Latin'},
   {'@language': 'fre', '@value': 'Latin Classique'}]},
 'member': [{'@id': 'urn:cts:latinLit:phi0959',
   '@type': 'Collection',
   'title': 'Ovid',
   'totalItems': 1}],
 'title': 'Classical Latin',
 'totalItems': 1}

### Ovid

See the `dts:extensions` property ? It seems that our collection has 2 `skos:prefLabel` in two languages : Classical Latin for English, Latin Classique in French !

But wait ! Another collection

In [5]:
Ovid_Collection  = requests.get(API_URI+ENDPOINTS["collections"]+"?id=urn:cts:latinLit:phi0959").json()
Ovid_Collection

{'@context': {'@vocab': 'https://www.w3.org/ns/hydra/core#',
  'cts': 'http://chs.harvard.edu/xmlns/cts/',
  'dts': 'https://w3id.org/dts/api#',
  'ns1': 'http://www.w3.org/2004/02/skos/core#'},
 '@id': 'urn:cts:latinLit:phi0959',
 '@type': 'Collection',
 'dts:extensions': {'cts:groupname': [{'@language': 'eng', '@value': 'Ovid'}],
  'ns1:prefLabel': [{'@language': 'eng', '@value': 'Ovid'}]},
 'member': [{'@id': 'urn:cts:latinLit:phi0959.phi006',
   '@type': 'Collection',
   'title': 'Metamorphoses',
   'totalItems': 1}],
 'title': 'Ovid',
 'totalItems': 1}

### Metamorphoses

Wait, it has again another collection. Let's go ! Let's see where this ends !

In [6]:
Metamorphoses_Collection  = requests.get(
    API_URI+ENDPOINTS["collections"]+"?id=urn:cts:latinLit:phi0959.phi006"
).json()
Metamorphoses_Collection

{'@context': {'@vocab': 'https://www.w3.org/ns/hydra/core#',
  'cts': 'http://chs.harvard.edu/xmlns/cts/',
  'dts': 'https://w3id.org/dts/api#',
  'ns1': 'http://www.w3.org/2004/02/skos/core#',
  'ns2': 'http://purl.org/dc/elements/1.1/'},
 '@id': 'urn:cts:latinLit:phi0959.phi006',
 '@type': 'Collection',
 'dts:extensions': {'cts:title': [{'@language': 'eng',
    '@value': 'Metamorphoses'}],
  'ns1:prefLabel': [{'@language': 'eng', '@value': 'Metamorphoses'}],
  'ns2:language': 'lat'},
 'member': [{'@id': 'urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1',
   '@type': 'Resource',
   'dts:citeDepth': 2,
   'dts:citeStructure': {'dts:citeStructure': [{'dts:citeType': 'line'}],
    'dts:citeType': 'line'},
   'dts:extensions': {'cts:description': [{'@language': 'eng',
      '@value': 'Alpheios Enhanced Metamorphoses'}],
    'cts:label': [{'@language': 'eng',
      '@value': 'Alpheios Enhanced Metamorphoses'}],
    'ns1:prefLabel': [{'@language': 'eng',
      '@value': 'Alpheios Enhanced

### Alpheios Edition of the Metamorphoses

Wait, the next one seems more complicated, let's request this single collection and read what's in there :

In [7]:
Metamorphoses_Edition_Collection  = requests.get(
    API_URI+ENDPOINTS["collections"]+"?id=urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"
).json()
Metamorphoses_Edition_Collection

{'@context': {'@vocab': 'https://www.w3.org/ns/hydra/core#',
  'cts': 'http://chs.harvard.edu/xmlns/cts/',
  'dts': 'https://w3id.org/dts/api#',
  'ns1': 'http://www.w3.org/2004/02/skos/core#',
  'ns2': 'http://purl.org/dc/elements/1.1/'},
 '@id': 'urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1',
 '@type': 'Resource',
 'dts:citeDepth': 2,
 'dts:citeStructure': {'dts:citeStructure': [{'dts:citeType': 'line'}],
  'dts:citeType': 'line'},
 'dts:extensions': {'cts:description': [{'@language': 'eng',
    '@value': 'Alpheios Enhanced Metamorphoses'}],
  'cts:label': [{'@language': 'eng',
    '@value': 'Alpheios Enhanced Metamorphoses'}],
  'ns1:prefLabel': [{'@language': 'eng',
    '@value': 'Alpheios Enhanced Metamorphoses'}],
  'ns3:language': 'lat'},
 'dts:passage': '/api/dts/document?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1',
 'dts:references': '/api/dts/navigation?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1',
 'title': 'Alpheios Enhanced Metamorpho

So, there is few things we can see:

- There is prefLabels again, and some value for `cts` ontology properties.
- More importantly, the `@type` is not `Collection` anymore ! This means the current Collection can actually be read, it's not only metadata. Good to know hmm ?
- You see the `dts:citeDepth` ? It means the text has two levels of citation. In the context of this collection, the data curator actually explicited them in `dts:citeStructure` :
    1. The first level has the name `poem`. This level has a second level:
        1. The second of the level inside poem has the name `line`

Now, we have two really interesting links, let's go see what's in there !

## What are the passages that I can single out in the Alpheios Edition of Metamorphoses ?

To reply to this long but quite clear title, there is only one thing to do : go to the `dts:references` URI we see here.

But wait, see the URI ? It's actually a simple construction :

- We use `navigation` from `ENDPOINTS`.
- We add the `@id` of the Resource we are interested in !

### All the Poems !

In [8]:
Ovid_Poems = requests.get(API_URI+Metamorphoses_Edition_Collection["dts:references"]).json()
Ovid_Poems

{'@context': {'@vocab': 'https://w3id.org/dts/api#',
  'hydra': 'https://www.w3.org/ns/hydra/core#'},
 '@id': '/api/dts/navigation?groupBy=1&level=1&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1',
 'citeDepth': 2,
 'citeType': 'poem',
 'hydra:member': [{'ref': '1'}],
 'level': 1,
 'passage': '/api/dts/document?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1{&ref}{&start}{&end}'}

Bummer ! There seems to be a single poem, here in `member`.

### Well, then all the lines of Poem 1!

But wait, didn't we say there was a second level ? To go see what's in there, we can simply go check the same URI, adding the ref we want to look into : `&ref=1`

In [9]:
Ovid_Poem_1_Lines = requests.get(API_URI+Metamorphoses_Edition_Collection["dts:references"]+"&ref=1").json()
Ovid_Poem_1_Lines

{'@context': {'@vocab': 'https://w3id.org/dts/api#',
  'hydra': 'https://www.w3.org/ns/hydra/core#'},
 '@id': '/api/dts/navigation?groupBy=1&level=1&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&ref=1',
 'citeDepth': 2,
 'citeType': 'line',
 'hydra:member': [{'ref': '1.1'},
  {'ref': '1.2'},
  {'ref': '1.3'},
  {'ref': '1.4'},
  {'ref': '1.5'},
  {'ref': '1.6'},
  {'ref': '1.7'},
  {'ref': '1.8'},
  {'ref': '1.9'},
  {'ref': '1.163'},
  {'ref': '1.164'},
  {'ref': '1.165'},
  {'ref': '1.166'},
  {'ref': '1.167'},
  {'ref': '1.168'},
  {'ref': '1.169'},
  {'ref': '1.170'},
  {'ref': '1.171'},
  {'ref': '1.172'},
  {'ref': '1.173'},
  {'ref': '1.174'},
  {'ref': '1.175'},
  {'ref': '1.176'},
  {'ref': '1.177'},
  {'ref': '1.178'},
  {'ref': '1.179'},
  {'ref': '1.180'},
  {'ref': '1.181'},
  {'ref': '1.182'},
  {'ref': '1.183'},
  {'ref': '1.184'},
  {'ref': '1.185'},
  {'ref': '1.186'},
  {'ref': '1.187'},
  {'ref': '1.188'},
  {'ref': '1.189'},
  {'ref': '1.190'},
  {'ref

That's quite a lot of references. All are lines, that's interesting. **We can even see that this edition is lacking the lines 10 to 162 !**

### TLDR : GROUPS !

But I don't want to check each line one by one. So, let's group them :

In [10]:
Ovid_Poem_1_Lines_Grouped = requests.get(API_URI+Metamorphoses_Edition_Collection["dts:references"]+"&ref=1&groupBy=20").json()
Ovid_Poem_1_Lines_Grouped

{'@context': {'@vocab': 'https://w3id.org/dts/api#',
  'hydra': 'https://www.w3.org/ns/hydra/core#'},
 '@id': '/api/dts/navigation?groupBy=20&level=1&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&ref=1',
 'citeDepth': 2,
 'citeType': 'line',
 'hydra:member': [{'end': '1.173', 'start': '1.1'},
  {'end': '1.193', 'start': '1.174'},
  {'end': '1.213', 'start': '1.194'},
  {'end': '1.233', 'start': '1.214'},
  {'end': '1.253', 'start': '1.234'},
  {'end': '1.273', 'start': '1.254'},
  {'end': '1.293', 'start': '1.274'},
  {'end': '1.313', 'start': '1.294'},
  {'end': '1.333', 'start': '1.314'},
  {'end': '1.353', 'start': '1.334'},
  {'end': '1.373', 'start': '1.354'},
  {'end': '1.393', 'start': '1.374'},
  {'end': '1.413', 'start': '1.394'},
  {'end': '1.433', 'start': '1.414'},
  {'end': '1.453', 'start': '1.434'},
  {'end': '1.473', 'start': '1.454'},
  {'end': '1.493', 'start': '1.474'},
  {'end': '1.513', 'start': '1.494'},
  {'end': '1.533', 'start': '1.514'},
  {'end'

### What's inside that group ?

So, we have group now. But how do I know, if there is missing lines, what's forming my group ? 

Answer : By doing the same query, but using the `start` and `end` parameter, specifying that I want things inside this range adding `&level=0`.

In [11]:
print(
    API_URI+Metamorphoses_Edition_Collection["dts:references"]+"&start=1.1&end=1.173&level=0"
)
Ovid_Poem_1_Lines_First_Group = requests.get(
    API_URI+Metamorphoses_Edition_Collection["dts:references"]+"&start=1.1&end=1.173&level=0"
).json()
Ovid_Poem_1_Lines_First_Group

http://texts.alpheios.net/api/dts/navigation?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.1&end=1.173&level=0


{'@context': {'@vocab': 'https://w3id.org/dts/api#',
  'hydra': 'https://www.w3.org/ns/hydra/core#'},
 '@id': '/api/dts/navigation?groupBy=1&end=1.173&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.1',
 'citeDepth': 2,
 'citeType': 'line',
 'hydra:member': [{'ref': '1.1'},
  {'ref': '1.2'},
  {'ref': '1.3'},
  {'ref': '1.4'},
  {'ref': '1.5'},
  {'ref': '1.6'},
  {'ref': '1.7'},
  {'ref': '1.8'},
  {'ref': '1.9'},
  {'ref': '1.163'},
  {'ref': '1.164'},
  {'ref': '1.165'},
  {'ref': '1.166'},
  {'ref': '1.167'},
  {'ref': '1.168'},
  {'ref': '1.169'},
  {'ref': '1.170'},
  {'ref': '1.171'},
  {'ref': '1.172'},
  {'ref': '1.173'}],
 'level': 2,
 'passage': '/api/dts/document?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1{&ref}{&start}{&end}'}

## Getting the text

Now that we can see what are the available passage, why not getting to the text passages ?

Let see... We build this the same way than the Navigation query ! But instead, we use `document` from the entry point !

### Getting an excerpt


In [12]:
Ovid_Poem_Text = requests.get(
    API_URI+Metamorphoses_Edition_Collection["dts:passage"]+"&start=1.1&end=1.173"
)
print(
    API_URI+Metamorphoses_Edition_Collection["dts:passage"]+"&start=1.1&end=1.173"
)
print(Ovid_Poem_Text.text)

http://texts.alpheios.net/api/dts/document?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.1&end=1.173
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text xml:lang="lat"><body xml:lang="lat" n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div1 type="Book" n="1" org="uniform" sample="complete"><l n="1"><w ana="1-1">In</w><w ana="1-2">nova</w><w ana="1-3">fert</w><w ana="1-4">animus</w><w ana="1-5">mutatas</w><w ana="1-6">dicere</w><w ana="1-7">formas</w></l><l n="2"><w ana="1-8">corpora</w>; <w ana="2-1">di</w>, <w ana="2-3">coeptis</w> (<w ana="2-5">nam</w> 
               <w ana="2-6">vos</w> 
               <w ana="2-7">mutastis</w> 
               <w ana="2-8">et</w> 
               <w ana="2-9">illas</w>)</l><l n="3"><w ana="2-11">adspirate</w><w ana="2-12">meis</w><w ana="2-13 2-14">primaque</w><w ana="2-15">ab</w><w ana="2-16">origine</

### Navigating in the Document Endpoint

That's nice ! But how do I know where to go next ! I am lost !

Wait no, because we thought about it ! Look at the headers !

In [13]:
print(Ovid_Poem_Text.headers["Link"])

</api/dts/collections?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1>; rel=collection, </api/dts/document?start=1.174&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&end=1.193>; rel=next, </api/dts/document?id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&ref=1>; rel=up, </api/dts/navigation?start=1.1&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&end=1.173>; rel=contents


See that sweet rel=next ? Let's go !

In [15]:
Ovid_Poem_Text_Next = requests.get(
    API_URI+"/api/dts/document?end=1.193&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.174"
)
print(API_URI+"/api/dts/document?end=1.193&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.174")
print(Ovid_Poem_Text_Next.text)

http://texts.alpheios.net/api/dts/document?end=1.193&id=urn%3Acts%3AlatinLit%3Aphi0959.phi006.alpheios-text-lat1&start=1.174
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:py="http://codespeak.net/lxml/objectify/pytype" py:pytype="TREE"><dts:fragment xmlns:dts="https://w3id.org/dts/api#"><text xml:lang="lat"><body xml:lang="lat" n="urn:cts:latinLit:phi0959.phi006.alpheios-text-lat1"><div1 type="Book" n="1" org="uniform" sample="complete"><l n="174"><w ana="11-4">caelicolae</w><w ana="11-5 11-6">clarique</w><w ana="11-7">suos</w><w ana="11-8">posuere</w><w ana="11-9">penates</w>.</l><l n="175"><w ana="12-1">Hic</w><w ana="12-2">locus</w><w ana="12-3">est</w>, <w ana="12-5">quem</w>, <w ana="12-7">si</w><w ana="12-8">verbis</w><w ana="12-9">audacia</w><w ana="12-10">detur</w>,</l><l n="176"><w ana="12-12">haud</w><w ana="12-13">timeam</w><w ana="12-14">magni</w><w ana="12-15">dixisse</w><w ana="12-16">Palatia</w><w ana="12-17">caeli</w>.</l><milestone ed="P" unit="para"/><l n="177"><w an